# Parsing Tabular Data

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse tabular files (CSV) files similar to the format established in
the [RTS-GMLC](github.com/gridmod/rts-gmlc/RTS_Data/SourceData) and create a `System` using
[PowerSystems.jl](github.com/NREL-SIIP/PowerSystems.jl)

### Environment
This notebook depends on the SIIPExamples.jl environment which is loaded by default

In [1]:
using Pkg
Pkg.status()

Project SIIPExamples v0.0.2
Status `~/Documents/repos/SIIPExamples.jl/Project.toml`
  [336ed68f] CSV v0.8.0
  [9961bab8] Cbc v0.7.1
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.22.0
  [2cd47ed4] InfrastructureSystems v1.0.4
  [b6b21f68] Ipopt v0.6.3
  [2535ab7d] JSON2 v0.3.2
  [98b081ad] Literate v2.7.0
  [f0f68f2c] PlotlyJS v0.14.0
  [5f7eddb3] PowerGraphics v0.6.1
  [e690365d] PowerSimulations v0.8.0
  [bcd98974] PowerSystems v1.0.2
  [9e3dc215] TimeSeries v0.19.1
  [f269a46b] TimeZones v1.5.0
  [0f1e0344] WebIO v0.8.15
  [ade2ca70] Dates
  [56ddb016] Logging
  [44cfe95a] Pkg
  [9a3f8284] Random
  [10745b16] Statistics
  [8dfed614] Test


### Dependencies

In [2]:
using SIIPExamples
using PowerSystems
using TimeSeries
using Dates

### Fetch Data
PowerSystems.jl links to some test data that is suitable for this example.
Let's download the test data

In [3]:
PowerSystems.download(PowerSystems.TestData; branch = "master") # *note* add `force=true` to get a fresh copy
base_dir = dirname(dirname(pathof(PowerSystems)));

### The tabular data format relies on a folder containing `*.csv` files and a `user_descriptors.yaml` file
First, we'll read the tabular data

In [4]:
RTS_GMLC_DIR = joinpath(base_dir, "data", "RTS_GMLC");
rawsys = PowerSystems.PowerSystemTableData(
    RTS_GMLC_DIR,
    100.0,
    joinpath(RTS_GMLC_DIR, "user_descriptors.yaml"),
    timeseries_metadata_file = joinpath(RTS_GMLC_DIR, "timeseries_pointers.json"),
    generator_mapping_file = joinpath(RTS_GMLC_DIR, "generator_mapping.yaml"),
)

┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:125
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/pow

PowerSystemTableData:
  directory:  /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/data/RTS_GMLC
  timeseries_metadata_file:  /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/data/RTS_GMLC/timeseries_pointers.json
  base_power:  100.0
  BRANCH:  120×14 DataFrame
  GENERATOR:  158×58 DataFrame
  STORAGE:  22×7 DataFrame
  RESERVE:  7×7 DataFrame
  DC_BRANCH:  1×60 DataFrame
  BUS:  73×15 DataFrame


### Create a `System`
Next, we'll create a `System` from the `rawsys` data. Since a `System` is predicated on a
forecast resolution and the `rawsys` data includes both 5-minute and 1-hour resolution
forecasts, we also need to specify which forecasts we want to include in the `System`.
The `time_series_resolution` kwarg filters to only include forecasts with a matching resolution.

In [5]:
sys = System(rawsys; time_series_resolution = Dates.Hour(1));
horizon = 24;
interval = Dates.Hour(24);
transform_single_time_series!(sys, horizon, interval);
sys

┌ Info: Unit System changed to DEVICE_BASE
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/base.jl:288
┌ Warning: Missing LOAD data.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:212
┌ Warning: User-defined column name Startup Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:1215
┌ Warning: User-defined column name Shutdown Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:1215
┌ Warning: User-defined column name Status at Start is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_system_table_data.jl:1215
┌ Warning: User-defined column name Time at Status is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/eF3Pv/src/parsers/power_

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,109
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,73
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
5,HVDCLine,DCBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
7,HydroEnergyReservoir,HydroGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,19
8,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,105
9,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,21


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*